In [12]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
import os, sys
import requests
import json
from openai import OpenAI
import httpx
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langgraph.checkpoint.sqlite import SqliteSaver
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults
from cfenv import AppEnv

# go one level up from cflangchainfolder/ to project root
sys.path.append(os.path.abspath(".."))
from cfutils import CFGenAIService

# connect to tavily search tool - use your tavily api key
os.environ['TAVILY_API_KEY']= "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"
tool = TavilySearchResults(max_results=2)

#define agent state
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
#memory = SqliteSaver.from_conn_string(":memory:")

print(type(tool))
print(tool.name)

class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
# Load CF environment
env = AppEnv()

# configure model
httpx_client = httpx.Client(http2=True, verify=False, timeout=10.0)
# Get bound service "gen-ai-qwen3-ultra"
chat_service = CFGenAIService("tanzu-gpt-oss-120b")
# List available models
models = chat_service.list_models()
for m in models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")

# construct chat_credentials
chat_credentials = {
    "api_base": chat_service.api_base + "/openai/v1",
    "api_key": chat_service.api_key,
    "model_name": models[0]["name"]
}

# Initialize LLM with credentials from cfenv
model = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],
    base_url=chat_credentials["api_base"],
    api_key=chat_credentials["api_key"],
    http_client=httpx_client
)
messages = [HumanMessage(content="Create a structured troubleshooting document based on the following aircraft equipment issues: The reported hydraulic‑leak issue is with **Landing Gear B")]
thread = {"configurable": {"thread_id": "1"}}
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, checkpointer=checkpointer)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json
- openai/gpt-oss-120b (capabilities: CHAT, TOOLS)


/home/vcap/app/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'genai-proxy.sys.tas-ndc.kuhn-labs.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[AIMessage(content='**Aircraft Equipment Troubleshooting Document  \nHydraulic‑Leak Issue – Landing‑Gear\u202fB (LG\u202fB)**  \n\n---\n\n### 1. Document Control\n| Item | Details |\n|------|---------|\n| **Document Title** | Troubleshooting Procedure – LG\u202fB Hydraulic Leak |\n| **Applicable Aircraft** | All aircraft equipped with a dual‑gear hydraulic system (e.g., B‑737NG, A320, B‑777) – adapt part numbers as necessary |\n| **Revision** | 1.0 |\n| **Author** | Maintenance Technical Writer – [Name] |\n| **Approved By** | Chief Engineer – [Name] |\n| **Effective Date** |\u202f[Date] |\n| **Distribution** | Maintenance, Engineering, Quality Assurance |\n\n---\n\n### 2. Issue Summary\n| Parameter | Description |\n|-----------|-------------|\n| **System** | Landing‑Gear (LG) hydraulic circuit – Gear\u202fB |\n| **Reported Symptom** | Visible hydraulic fluid leak at or near the LG\u202fB assembly; low‑pressure warning on the EICAS/ECAM; possible “Gear\u202fB not down & locked” indicati